In [32]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

In [311]:
# Прочитаем входные файлы, объединим их
stations = pd.read_excel("C:/Users/user/Desktop/Геофак учёба/1 МАГА/Пайтон у Энтина/Проект/Парсер станций метро/subway.xlsx").drop(columns="Unnamed: 0")
houses =  pd.read_excel("C:/Users/user/Desktop/Геофак учёба/1 МАГА/Пайтон у Энтина/Проект/Парсер недвижимости/flats.xlsx").drop(columns="Unnamed: 0")
poi_counts = pd.DataFrame(gpd.read_file("C:/Users/user/Desktop/Геофак учёба/1 МАГА/Пайтон у Энтина/Проект/Подсчёт POI/fifteen_minutes_iso_counted.gpkg").drop(columns="geometry")).drop(columns="walk_time").drop_duplicates(subset="name", keep='first', inplace=False)

stations_with_poi = pd.merge(stations, poi_counts, how="inner", left_on="station", right_on="name").drop_duplicates(subset="station", keep='first', inplace=False).drop(columns="name")

houses["station"] = houses["station"].apply(lambda x: "Технологический институт" if re.match(r"Технологический ин-т.*", x) else("Площадь Александра Невского" if re.match(r"Площадь А. Невского.*", x) else x))
houses["rooms"] = houses["rooms"].apply(lambda x: re.sub(" в апарт-отеле", "", x))
poi_counts

,name,"food, on km2","greenery, overlap","leisure, on km2","shops, on km2","tourism, on km2"
0,Девяткино,44.3,0.08,6.0,121.4,0.5
1,Гражданский проспект,14.7,0.09,4.4,81.8,3.6
2,Академическая,25.8,0.11,4.9,107.5,2.7
3,Политехническая,10.9,0.23,5.1,15.3,2.6
4,Площадь Мужества,14.8,0.21,4.6,26.2,3.4
...,...,...,...,...,...,...
67,Бухарестская,9.5,0.14,2.4,33.4,6.0
68,Международная,12.0,0.17,2.3,71.5,1.2
69,Проспект Славы,13.1,0.28,3.0,53.5,6.0
70,Дунайская,5.9,0.34,2.8,28.5,1.6


In [312]:
# Удалим редко встречающиеся параметры
houses = houses[~(houses.rooms=="4-к. квартира") & ~(houses.walk.isin(["21–30 мин.", "от 31 мин."]))]
houses

,rooms,area,beds,price,station,walk
0,Квартира-студия,20,1,2000,Сенная площадь,до 5 мин.
1,Квартира-студия,25,1,2399,Обводный канал,11–15 мин.
2,Квартира-студия,28,2,2000,Старая деревня,11–15 мин.
3,Квартира-студия,14,3,1400,Площадь Восстания,до 5 мин.
4,Квартира-студия,28,2,1800,Проспект Просвещения,6–10 мин.
...,...,...,...,...,...,...
4918,Квартира-студия,25,2,2500,Маяковская,до 5 мин.
4919,Квартира-студия,30,2,1700,Ладожская,11–15 мин.
4920,1-к. квартира,37,2,2100,Пионерская,до 5 мин.
4921,1-к. квартира,45,2,3500,Достоевская,до 5 мин.


In [445]:
center_point = "Невский проспект"

room_map = {
    "Квартира-студия":0,
    "1-к. квартира":1,
    "2-к. квартира":2,
    "3-к. квартира":3}

walk_map = {
    "до 5 мин.":5,
    "6–10 мин.":10,
    "11–15 мин.":15
}

houses_cat = pd.DataFrame.copy(houses, deep=True)
houses_cat["rooms"] = houses_cat["rooms"].map(room_map)
houses_cat["walk"] = houses_cat["walk"].map(walk_map)
houses_cat = pd.merge(left=houses_cat, right=stations_with_poi, how="inner", on="station").drop(columns=["depth", "img", "lat", "lon", "district", "line"])
houses_cat.insert(9, "time_to_center", None)

walk_out_from_center_point = stations[lambda x: x["station"]==center_point]["entrance"].tolist()[0]
houses_cat["time_to_center"] = houses_cat["time"] + houses_cat["entrance"] + walk_out_from_center_point
houses_cat[lambda x: x["station"]==center_point]["time_to_center"] = houses_cat[lambda x: x["station"]==center_point]["entrance"].apply(lambda x: 0)
houses_cat.drop(columns=["entrance", "time"], inplace=True)
houses_cat = houses_cat[[
    "price",
    "rooms",
    "area",
    "beds",
    "walk",
    "time_to_center",
    "food, on km2",
    "leisure, on km2",
    "shops, on km2",
    "tourism, on km2",
    "greenery, overlap",
    "station"]]

# Разделяем параметры районов по квантилям
place_metrics = poi_counts.columns.tolist()
place_metrics.remove("name")
for element in place_metrics:
    houses_cat[element] = pd.qcut(houses_cat[element], 5, [1, 2, 3, 4, 5])
    
# Сборка и очистка датафрейма для поступления в модель
houses_cat.rename(columns={el:el.split(",")[0] for el in houses_cat.columns.tolist()}, inplace=True)
houses_cat.dropna(inplace=True)
for column in houses_cat.columns.tolist()[:-1]:
    houses_cat[column] = houses_cat[column].astype("str")
    houses_cat[column] = houses_cat[column].apply(lambda x: x.replace(",","."))
    houses_cat[column] = houses_cat[column].apply(lambda x: int(round(float(x))))
houses_to_learn = houses_cat.copy()
houses_to_learn.drop(columns="station", inplace=True)

houses_to_learn


C:\Users\user\AppData\Local\Temp\ipykernel_23468\3645578842.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses_cat[lambda x: x["station"]==center_point]["time_to_center"] = houses_cat[lambda x: x["station"]==center_point]["entrance"].apply(lambda x: 0)


,price,rooms,area,beds,walk,time_to_center,food,leisure,shops,tourism,greenery
0,2000,0,20,1,5,14,4,4,5,4,2
1,2399,0,25,1,15,24,3,3,3,3,4
2,1400,0,14,3,5,22,5,5,5,5,1
3,1800,0,28,2,10,33,2,3,3,1,2
4,1500,0,25,2,5,18,4,4,4,4,2
...,...,...,...,...,...,...,...,...,...,...,...
4855,2500,0,25,2,5,19,5,5,5,5,1
4856,1700,0,30,2,15,32,1,1,1,1,1
4857,2100,1,37,2,5,25,2,1,2,1,5
4858,3500,1,45,2,5,23,5,5,5,5,2


In [527]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

def recommend_stations(stations=houses_cat["station"], table=houses_cat.copy(), subway_df=stations, **params):

    ''' 
    Данная функция рекомендует пользователю станции метро в Санкт-Петербурге 
    для посуточной аренды жилья на основании его запросов по серии параметров.\n\n
    Аргументы функции (обязательны минимум 4 аргумента):\n
    price | int\n
    rooms | int\n
    area | int\n
    beds | int\n
    walk | int\n
    time_to_center | int\n
    food | int\n
    shops | int\n
    tourism | int\n
    greenery | int\n
    leisure | int\n
    '''

    # Пропишем ошибки, которые будем вызывать при нарушении работы функции
    param_value_error = "Недопустимое значение парaметра"
    param_type_error = "Недопустимый тип парaметра."
    not_enough_params_error = "Отклонение операции. Чрезмерно низкое число парметров."
    unknow_param_error = "Присутствуют неизвестные параметры."

    # Зададим минимальное число входных параметров
    min_params = 4

    # Пропишем число рекоммендуемых станций
    number_of_recommendated_stations = 5

    # Пропишем функции проверок входящих параметров
    def check_type(value): # Проверяет целочисленность аргумента
        if type(params[value]) != int:
            raise TypeError(f"{param_type_error}")
    def check_value(value): # Проверяет вхождение аргумента в диапазоны изначальной таблицы
        if (params[value]<table[value].min()) or (params[value]>table[value].max()):
            raise ValueError(f"{param_value_error} {value}.")
    def check_existense(value): # Проверяет существование аргумента в таблице
        if value not in table.columns.tolist():
            raise KeyError(f"{unknow_param_error}")
    
    def set_param(value):
        check_type(value)
        check_value(value)
        check_existense(value)
        output_series = pd.concat((pd.Series(params[value]), table[value]), axis=0).reset_index(drop=True)
        return output_series
        
    # В данный список будем записывать ряды данных для обучения модели
    params_series_list = [set_param(param) for param in params]
    
    # Создаем DataFrame с выборкой для изучения, где первой строкой будет являться запрос к базе. Проверяем длину и масштабируем.
    params_dict = {name:param.tolist() for name, param in zip(
        table.columns.tolist(),
        params_series_list
    )}
    train_df = pd.DataFrame(params_dict)
    if len(train_df.columns.tolist())<min_params:
        raise ValueError(f"{not_enough_params_error}")
    scaler = StandardScaler()
    train_df = pd.DataFrame(scaler.fit_transform(train_df), columns=train_df.columns.tolist())

    # Обучаем модель
    knn = NearestNeighbors(metric = 'cosine',
                       algorithm = 'brute',
                       n_neighbors = number_of_recommendated_stations*30,
                       n_jobs = -1)
    knn.fit(train_df)

    # Получаем пары "индекс-косинусное сходство"
    recommendations = number_of_recommendated_stations*15
    request = train_df.iloc[0:1,:]
    distances, indices = knn.kneighbors(request, n_neighbors = recommendations + 1)
    indices_list = indices.squeeze().tolist()
    distances_list = distances.squeeze().tolist()
    indices_distances = list(zip(indices_list, distances_list))[1:]

    # Получаем список рекомендуемых станций и средние меры сходства для них
    indices_distances_stations = []
    for result in indices_distances:
        result = list(result)
        result.append(stations.iloc[result[0]-1])
        indices_distances_stations.append(result)
    indices_distances_stations_df = pd.DataFrame({
        "indices":[el[0] for el in indices_distances_stations],
        "distances":[el[1] for el in indices_distances_stations],
        "station":[el[2] for el in indices_distances_stations],
    })
    indices_distances_stations_df = indices_distances_stations_df.groupby("station").mean("distances")["distances"].apply(lambda x: f"{round((1-x)*100)}%")
    recommended_stations_list = []
    for result in indices_distances_stations:
        recommended_stations_list.append(result[2])
        recommended_stations_list = list(set(recommended_stations_list))
        if len(recommended_stations_list) == number_of_recommendated_stations:
            break

    # Настраиваем финальный датафрейм с рекомендуемыми станциями
    mean_price_by_m2 = table.groupby("station").mean()
    mean_price_by_m2["mean_price_by_m2"] = round(mean_price_by_m2["price"] / mean_price_by_m2["area"])
    mean_price_by_m2 = mean_price_by_m2["mean_price_by_m2"]
    lines_list = [subway_df[subway_df["station"]==station]["line"].tolist()[0].split()[0] for station in recommended_stations_list]
    district_list = [subway_df[subway_df["station"]==station]["district"].tolist()[0].split()[0] for station in recommended_stations_list]
    ride_time_list = [subway_df[subway_df["station"]==station]["time"].tolist()[0] for station in recommended_stations_list]
    mean_price_list = [int(mean_price_by_m2.loc[station]) for station in recommended_stations_list]
    similarity = [indices_distances_stations_df[station] for station in recommended_stations_list]
    recommendations_df = pd.DataFrame({
        "Станция метро":recommended_stations_list,
        "Линия":lines_list,
        "Район города":district_list,
        f"Время до ст. {center_point}":ride_time_list,
        "Цена за 1 м2, руб.":mean_price_list,
        "Сходство с запросом":similarity
    })
    return recommendations_df.sort_values("Сходство с запросом", ascending=False).reset_index(drop=True) 
    

    


In [528]:
recommend_stations(
    price = 2600,
    rooms = 0,
    area = 23,
    beds = 1,
    walk = 12,
    food = 2,
    shops = 3,
    tourism = 3,
    greenery = 2,
    leisure = 3
)

,Станция метро,Линия,Район города,Время до ст. Невский проспект,"Цена за 1 м2, руб.",Сходство с запросом
0,Проспект Просвещения,Московско-Петроградская,Выборгский,23,73,77%
1,Спортивная,Фрунзенско-Приморская,Петроградский,15,94,76%
2,Фрунзенская,Московско-Петроградская,Адмиралтейский,10,78,76%
3,Чкаловская,Фрунзенско-Приморская,Петроградский,17,101,73%
4,Парнас,Московско-Петроградская,Выборгский,26,88,73%
